# 마이데이터 투자탭 데모 

## Import libraries

In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly

import pandas as pd
import numpy as np

import scipy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS

import pickle

In [ ]:
import FinanceDataReader as fdr
import quantstats as qs

In [ ]:
## custom libs

from korquanttools.pricevolume.loader import KRXPriceDM
from korquanttools.pricevolume.utils import DateUtil
from korquanttools.pricevolume.config import PathConfig

## Import dataset

In [ ]:
# Global parameters

START = 20140101
END = 20221202

WINDOW = 252 # trading day 기준
# WINDOW = 60 # trading day 기준
# WINDOW = 20 # trading day 기준

In [ ]:
# Init data module
pricevolume = KRXPriceDM(START, END)
pricevolume.get_info()


        * DM name: KRX_pricevolume
        * DM description: Basic price-volume data imported from KRX website & NAVER finance. Has KOSPI, KOSDAQ, KONEX stocks.
        * birthday: 20211203
        * DM period: 19990101 ~ 
        * Available data: ['lv1', 'open', 'high', 'low', 'close', 'volume', 'dollarvolume', 'marketcap']
        


### 수익률

In [ ]:
## stock return
return_df = pd.read_pickle(PathConfig.cache_path / f"temp_return_{START}_{END}")

In [ ]:
holidays = return_df.isnull().all(axis=1)
tradingdays = ~holidays

holidays = holidays.index[holidays]
tradingdays = tradingdays.index[tradingdays]

tradingdays

DatetimeIndex(['2014-01-02', '2014-01-03', '2014-01-06', '2014-01-07',
               '2014-01-08', '2014-01-09', '2014-01-10', '2014-01-13',
               '2014-01-14', '2014-01-15',
               ...
               '2022-11-21', '2022-11-22', '2022-11-23', '2022-11-24',
               '2022-11-25', '2022-11-28', '2022-11-29', '2022-11-30',
               '2022-12-01', '2022-12-02'],
              dtype='datetime64[ns]', name='trdDd', length=2190, freq=None)

In [ ]:
return_df = return_df.loc[tradingdays, :].copy()

### 시가총액

In [ ]:
# Data for 2nd Factor: SMB (Small Minus Big)

marcap_df = pricevolume.get_data("marketcap")

In [ ]:
marcap_df = marcap_df.astype(float) # object로 되어있었음

In [ ]:
marcap_df = marcap_df.loc[tradingdays, :].copy()

### Grouping

In [ ]:
with open('INDUSTRY_NAME2CODE.pickle', 'rb') as handle:
    INDUSTRY_NAME2CODE = pickle.load(handle)

In [ ]:
INDUSTRY_CODE2NAME = {v:k for k,v in INDUSTRY_NAME2CODE.items()}

In [ ]:
industry_df = pd.read_pickle('krx_industry_df_20140101_20221202.pickle')

In [ ]:
with open('sid2name.pkl', 'rb') as p:
    sid2name = pickle.load(p)

## 업종별 수익률

In [ ]:
DATE_BEFORE = -1

In [ ]:
last_return_v = return_df.iloc[DATE_BEFORE]

last_marcap_v = marcap_df.iloc[DATE_BEFORE]
last_group_v = industry_df.iloc[DATE_BEFORE]

In [81]:
last_weight_v = marcap_df.iloc[DATE_BEFORE] / np.nansum(marcap_df.iloc[DATE_BEFORE])

In [82]:
last_weighted_return_v = last_return_v * last_weight_v

In [83]:
last_lv1_df = pd.DataFrame(data={
    'ret': last_return_v,
    'marcap': last_marcap_v,
    'weight': last_weight_v,
    'weighted_return': last_weighted_return_v,
    'group_idx': last_group_v,
})

In [84]:
group_tree_df = last_lv1_df[['group_idx', 'weighted_return', 'marcap']].groupby(by='group_idx').sum()

In [85]:
group_tree_df.reset_index(drop=False, inplace=True)

In [86]:
group_tree_df['group_name'] = group_tree_df['group_idx'].apply(lambda g_idx: INDUSTRY_CODE2NAME[g_idx])

In [87]:
group_tree_df.columns

Index(['group_idx', 'weighted_return', 'marcap', 'group_name'], dtype='object')

In [88]:
chart_name = '업종별 수익률 현황'

treemap_fig = px.treemap(
    group_tree_df, 
    title='chart_name',
    path=[
        px.Constant('KOSPI & KOSDAQ'),
        'group_name',
    ],
    values='marcap',
    color='weighted_return',
    # hover_data=[],
    color_discrete_sequence=,
    # color_continuous_scale='RdBu_r',
    # color_continuous_midpoint=0,
)

treemap_fig.update_layout(margin=dict(t=50, l=25, b=25))
treemap_fig.show()
